In [1]:
from scipy.linalg import eigh, eigvalsh
from scipy.sparse import coo_matrix
import numpy as np
import networkx,math,sys
import time
from scipy.stats import unitary_group, ortho_group
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

def psdrF(mat,r,d=None):
    if d is None:
        d = len(mat)
    vas,ves = eigh(mat)
    vas[vas<0]=0
    pvas=np.diag(vas*np.array([0]*(d-r)+[1]*r))
    ves = np.matrix(ves)
    return np.matmul(np.matmul(ves,pvas),ves.H)

def normF(mat,adj,d=None):
    if d is None:
        d = len(mat)
    return np.multiply(mat,adj) + 1*np.identity(d)

def ranH(d):
    Mr = np.random.rand(d,d)
    Mi = np.random.rand(d,d)
    M = (Mr+Mr.T)/2 + 1j*(Mi-Mi.T)/2
    return np.matrix(M)

def dimQ(ghf,r,t): # k is the rank of the projectors; r is the rank of the matrix, i.e., the dimension of the vectors; t is the number of steps in the see-saw method in each monitored period
    if isinstance(ghf,str):gh = networkx.from_graph6_bytes(ghf.encode())
    elif isinstance(ghf,list) or isinstance(ghf,np.matrix):gh = networkx.Graph(ghf)
    else:return('Accepted types:str of graph6 format,list of edges, np.matrix of adjacency matrix')
    d = len(gh)
    adjc = 1-np.matrix(networkx.adjacency_matrix(gh).A)-np.identity(d)
    iterM = ranH(d)
    tmp0 = d
    tmp1 = 2*d
    start0 = time.time()
    start = start0
    while abs(tmp1-tmp0)>1e-9 and start-start0<30:
        newtime = time.time()
        if newtime-start>3:
            iterM = iterM + ranH(d)/100
            start = newtime
        for i in range(t): # t is the number of steps in the see-saw method in each monitored period
            iterM = psdrF(iterM,r,d)
            iterM = normF(iterM,adjc,d)
        vas = eigvalsh(iterM,eigvals=(0,d-r-1))
        tmp0 = tmp1
        tmp1 = sum(abs(vas))
    #return [abs(tmp1-tmp0)<1e-9,tmp1<1e-9]
    if tmp1<1e-9:
        return 'True'
    else:
        return 'Undetermied'

def dimQM(ghf,r,lt): # lt is the number of rounds
    res = []
    for i in range(lt):
        tmp = dimQ(ghf,r,20)
        if tmp[0]:res.append(tmp[1])
        if tmp[1]<1e-6:return [len(res),tmp]
        if len(res)>=5:return [len(res),min(res)]
    return [len(res),min(res+[1000])]



In [2]:
from newdim import *

gh = [[0,1],[1,2],[2,3],[3,4],[0,4]] #edge format
gh = 'Dhc' #graph6 format
res = dimQ(gh,3,10) # graph, orthogonal rank to check, length of iteration in each small step (the larger the more accurate, then more time)
# The result 'Ture' means that we have a orthogonal representation with such a rank, 'Undetermined' means that it cannot be decided by this routine. We can always increase the targeted orthogonal rank to obtain the result 'True'.
print(res)

True


In [3]:
import csv
from csv import reader
from time import time

In [6]:
t0=time()
mini=1000

for vertices in range(19,31):
    mini=1000
    with open('vtc'+str(vertices)+'t','r') as read_obj:
        csv_reader = reader(read_obj,delimiter='\t')
        # header = next(csv_reader)
        l=0
        # Check file as empty
        # if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader: 
            # row variable is a list that represents a row in csv
            l=l+1
            lin=[row[0]]
            for s in row[1:]:
                n=eval(s)
                lin=lin+[n]
            # print(lin)
            d=int(vertices/lin[3])
            #print(d)
            if d>2:
                res = dimQ(row[0],d,10)
                while res != 'True':
                    d=d+1
                    res = dimQ(row[0],d,10)
                eta=2*lin[1]/(lin[1]+(vertices/d))
                if eta <= mini:
                    mini=min(mini,eta)
                    print(l,' ',lin,' ',eta,' ',d)
    if mini > 1:
        mini=1
    print(vertices,'\t',mini,'\t',l)
                # row variable is a list that represents a row in csv

t1=time()

5   ['R}hPOSS????@?C?B_AwEO@K??qGAW_', 6, 6.23943886, 6.33333333]   60/49   5
9   ['R}ecZ@OH?o`W@gOWcI_`p?hkHL?GuG', 4, 5.24042975, 6.33333333]   48/43   6
15   ['R~zLaWqCgY?U@Q??_?g@j?Qw?Jg?jg', 4, 4.58729963, 4.75]   48/43   6
23   ['R}nDCA?Vh}H\\L[BwA{AXDKpT?jhAjg', 5, 5.67191319, 6.33333333]   40/39   4
33   ['R}j\\}QpqLaLn[}R[E\\IwdMbSEzwMzo', 4, 4.2679217, 4.75]   40/39   5
38   ['R}n~uUssLQLx\\kA}AnIHtGvSGnwanw', 4, 4.2679217, 4.75]   40/39   5
39   ['R~yKmu{odGlu\\qEmbjihpIuTBnXJlo', 4, 4.39834714, 4.75]   40/39   5
46   ['R}nLnVsqli~O~?V~f~q^{L~dJ}hj}g', 4, 4.14187028, 4.75]   40/39   5
19 	 1 	 58


SyntaxError: unexpected EOF while parsing (<string>, line 0)

In [7]:
print(vertices,'\t',mini,'\t','\t',l)

20 	 1000 	 	 1


In [12]:
t0=time()
mini=1000

for vertices in range(20,31):
    mini=1000
    with open('vtc'+str(vertices)+'t367plus','r') as read_obj:
        csv_reader = reader(read_obj,delimiter='\t')
        # header = next(csv_reader)
        l=0
        # Check file as empty
        # if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader: 
            # row variable is a list that represents a row in csv
            l=l+1
            lin=[row[0]]
            for s in row[1:]:
                n=eval(s)
                lin=lin+[n]
            # print(lin)
            d=int(vertices/lin[3])
            #print(d)
            if d>2:
                res = dimQ(row[0],d,10)
                while res != 'True':
                    d=d+1
                    res = dimQ(row[0],d,10)
                eta=2*lin[1]/(lin[1]+(vertices/d))
                if eta <= mini:
                    mini=min(mini,eta)
                    print(l,' ',lin,' ',eta,' ',d)
    if mini > 1:
        mini=1
    print(vertices,'\t',mini,'\t',l)
                # row variable is a list that represents a row in csv

t1=time()

1   ['S}r\\|AxsTQMnF[YiBNJKeEilBLXG|gMf[', 4, 5, 5]   1   5
2   ['S}y|ffGhMDH~MYNc?~bocDq{EtWbViOvk', 4, 4.47213595, 5]   1   5
3   ['S}y|vbodeAF}K^Bu@vAFUPYgYXxWrq@N{', 4, 4.47213595, 5]   1   5
9   ['S~jN]epjMwIlNLYsAbobfArX@lXDbw[T{', 4, 4.18753031, 5]   1   5
10   ['S~nM^BPoE?e~L~B]@^J{BA^CHyXkbXkR[', 4, 4.47213595, 5]   1   5
12   ['S~nmuitycSNZ\\WDY@VGFU@M|G[xIFwC~k', 4, 4.4620272, 5]   1   5
14   ['S~u~UjdmcSNV\\WAe_xOhV?]{g\\gEziJB{', 4, 4.42920201, 5]   1   5
17   ['S~zEDIaqD_B~F}F}HN`kxRYTJiPlbXmdW', 4, 4.47213595, 5]   1   5
24   ['Ss~fFnofK{]OV~ZmDv?mx@vXwdzbJoR{{', 4, 5, 5]   1   5
34   ['St~enfsjStX?XvTnDlgvkIng\\\\ZEZihls', 4, 4.07948002, 5]   1   5
35   ['St~lkni{MixWVxZVErHPyPrKYuyBnpbT{', 4, 4.12022659, 5]   1   5
42   ['Suz|KrYlVS]G[zX|b^buoQ{kE}owyxJZ[', 4, 4.28280096, 5]   1   5
44   ['Svmud^MpV_|OR^E~lvBNgIfk[mpfbYiY{', 4, 4.30725123, 5]   1   5
46   ['Svvv]YN{tBV@Zy\\lAVgZYLTKJ]z?~qpT{', 4, 4.15047756, 5]   1   5
48   ['Svz~vbgw^BRo^}O^CN_Jz?~W]Tw{Zo{F{', 4, 4.47

FileNotFoundError: [Errno 2] No such file or directory: 'vtc21t367plus'

In [11]:
print(l)


368


In [13]:
t0=time()
mini=1000

for vertices in range(21,31):
    mini=1000
    with open('vtc'+str(vertices)+'t','r') as read_obj:
        csv_reader = reader(read_obj,delimiter='\t')
        # header = next(csv_reader)
        l=0
        # Check file as empty
        # if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader: 
            # row variable is a list that represents a row in csv
            l=l+1
            lin=[row[0]]
            for s in row[1:]:
                n=eval(s)
                lin=lin+[n]
            # print(lin)
            d=int(vertices/lin[3])
            #print(d)
            if d>2:
                res = dimQ(row[0],d,10)
                while res != 'True':
                    d=d+1
                    res = dimQ(row[0],d,10)
                eta=2*lin[1]/(lin[1]+(vertices/d))
                if eta <= mini:
                    mini=min(mini,eta)
                    print(l,' ',lin,' ',eta,' ',d)
    if mini > 1:
        mini=1
    print(vertices,'\t',mini,'\t',l)
                # row variable is a list that represents a row in csv

t1=time()

11   ['TuiCAStQ`g?rBCCaK?WH_?qD@@P@CWOJSOEs', 6, 7, 7]   32/23   8
12   ['TujLbPpB?cGgGgATADOCS@D??Aw?Iw?pW?XR', 6, 6.32974879, 7]   32/23   8
13   ['TunDAGaOpH@hCkS?D?GE`@IOGCo_EWBBo?]E', 6, 6.31046232, 7]   4/3   7
15   ['T{fLb?WP@CAQBAOTCHOPSCL?ALOGdOB@S@`T', 6, 6.32974879, 7]   4/3   7
16   ['T}iC?s[SHQgXHKA_A_AGFG`WCHGC`_AEK?cf', 6, 6.32974879, 7]   4/3   7
17   ['T}iCA[vS`W@_C_Ogcg_OPCCO?Kg?eg?[s?U[', 6, 6.67280857, 7]   4/3   7
18   ['T}iCAoxA_gg`GcB?AOACFH@WAhGI`_@lC@NC', 6, 6.32971318, 7]   4/3   7
19   ['T}iCAoxB_k@OD?P?cOaCPHCS?m_Am?@dS@LS', 6, 6.32974879, 7]   4/3   7
20   ['T}iCAoxP`K@PDCB?AOACTHDO?l_Af?@ES@Ct', 6, 6.32974879, 7]   4/3   7
22   ['T}iCIoxP@G@OC`@K?hGDE?pH?SX?SwOy?OY`', 6, 6.32974879, 7]   4/3   7
24   ['T~zLaWqCgY?U@Q????G?F?@[?sgAU_?Vc?Tk', 5, 5.15147471, 5.25]   20/17   6
26   ['TunDCA@BglHTLKWqkiae_JQCLFOeLQPaSogt', 6, 6.29150262, 7]   16/15   4
34   ['T}iCCA?Fw~HsLpQwEkAt@NOPAIhGii@dS`LT', 6, 6.54133625, 7]   16/15   4
35   ['T}iCCA?Fw~HsRoLWe[a{?MWDCHi

LinAlgError: Internal Error.

In [14]:
t0=time()
mini=1000

for vertices in range(22,31):
    mini=1000
    with open('vtc'+str(vertices)+'t','r') as read_obj:
        csv_reader = reader(read_obj,delimiter='\t')
        # header = next(csv_reader)
        l=0
        # Check file as empty
        # if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader: 
            # row variable is a list that represents a row in csv
            l=l+1
            lin=[row[0]]
            for s in row[1:]:
                n=eval(s)
                lin=lin+[n]
            # print(lin)
            d=int(vertices/lin[3])
            #print(d)
            if d>2:
                res = dimQ(row[0],d,10)
                while res != 'True':
                    d=d+1
                    res = dimQ(row[0],d,10)
                eta=2*lin[1]/(lin[1]+(vertices/d))
                if eta <= mini:
                    mini=min(mini,eta)
                    print(l,' ',lin,' ',eta,' ',d)
    if mini > 1:
        mini=1
    print(vertices,'\t',mini,'\t',l)
                # row variable is a list that represents a row in csv

t1=time()

6   ['U}hPOSS????@?C???@g?e??s?DgEO?H_??q@?R?_', 7, 7.25131314, 7.33333333]   70/57   5
18   ['U~rH`cKBGE?X?K?K_@_?X??o?BG?@_?@c??N??Bw', 5, 5.38630291, 5.5]   70/57   7
45   ['U}vc?ocHGcGPGbAaBG?CR@@_@pO@ag?kO@_h?OIw', 5, 5.38630291, 5.5]   70/57   7
47   ['U}zS?ggIWdagG_AP?VA?A?agCAOCKWAIo?aU??\\W', 5, 5.38630291, 5.5]   70/57   7
48   ['U~zLaWqCgY?U@Q????G?B??{?Do@ggATO?Jh?DT_', 5, 5.38630291, 5.5]   70/57   7
82   ['U}uTN@IBiBCVEC@ccG@AHE?gS@wGBgATO?kR?Aqw', 5, 5.38630291, 5.5]   70/57   7
95   ['U~zLc@KKg\\av?o@O@GGaCOO[`Aw?M_?VC?QV?GYw', 4, 4.81644749, 5.5]   32/27   8
129   ['U}iCCA?RxnBKUaNCBS?y?OwSdaaDIhILTALsHDm?', 6, 6.70930807, 7.33333333]   24/23   4
145   ['U}nMTao^y[HoKWKW?p_KW?Wo@b??Xw?Xw?B~??~o', 6, 6.9035698, 7.33333333]   24/23   4


LinAlgError: Internal Error.

In [8]:
t0=time()
mini=1000

for vertices in range(24,25):
    mini=1000
    with open('vtc'+str(vertices)+'tB','r') as read_obj:
        csv_reader = reader(read_obj,delimiter='\t')
        # header = next(csv_reader)
        l=0
        # Check file as empty
        # if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader: 
            # row variable is a list that represents a row in csv
            l=l+1
            lin=[row[0]]
            for s in row[1:]:
                n=eval(s)
                lin=lin+[n]
            # print(lin)
            d=int(vertices/lin[3])
            #print(d)
            if d>2:
                res = dimQ(row[0],d,10)
                while (res != 'True' and lin[1]*d < vertices):
                    d=d+1
                    res = dimQ(row[0],d,10)
                eta=2*lin[1]/(lin[1]+(vertices/d))
                if eta <= mini:
                    mini=min(mini,eta)
                if eta < 1:
                    print(l,' ',lin,' ',eta,' ',d)
    if mini > 1:
        mini=1
    print(vertices,'\t',mini,'\t',l)
                # row variable is a list that represents a row in csv

t1=time()

KeyboardInterrupt: 

In [7]:
print(l,'\t',vertices,'\t',mini,'\t',lin)


8 	 24 	 1 	 ['W}~t|R@wE_wKD]DxNo?T\\@lkBx`OypRLGB\\dTHPwisvDBZq', 4, 4.6771999, 6]


In [6]:
print(lin)

['W}jLeA_RxniWJGBGAWAEPHKOGaGaG_OESOCt??zg?Jm_?{{', 6, 6.8801856, 8]


In [17]:
print(lin)

['U}j\\}u{wvHXCkPSndjo\\|Dvpwmjqmb{r?Bi~@\\Vw', 4, 4.59536023, 5.5]
